In [1]:
import numpy as np
import pandas as pd
import open3d
import copy
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import utils.helpers as helpers
import utils.fread as fread
import utils.FCGF as FCGF
import utils.registration as registration
import scipy

from sklearn.decomposition import PCA

from utils.depth_camera import DepthCamera

plt.style.use("ggplot")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_dir = "data/raw_data"
experiment = "exp_8"
trial = "trial_2"
subject = "subject-1"
sequence = 2
voxel_size = 0.03

file_name = f"{experiment}__{trial}__{subject}__{sequence:02d}"
pose_file = os.path.join("data/trajectories/groundtruth", experiment, f"{file_name}.pose.npz")

motion_dir = os.path.join(data_dir, experiment, trial, subject, f"{sequence:02d}", "motion")
sequence_dir = os.path.join(data_dir, experiment, trial, subject, f"{sequence:02d}", "frames")

features_dir = os.path.join("data/features", experiment, trial, str(voxel_size), subject, f"{sequence:02d}")

In [3]:
sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")

num_frames = len(sequence_ts)

In [4]:
local_pcds = []

for t in tqdm.trange(num_frames):
    pcd = FCGF.get_features(os.path.join(features_dir, f"{sequence_ts[t]}.secondary.npz"), pcd_only=True)
    # pcd = helpers.downsample(pcd, voxel_size)
    local_pcds.append(pcd)

100%|██████████| 490/490 [00:06<00:00, 74.70it/s] 


In [30]:
local_t = np.load(pose_file)["local_t"]
trajectory_t = np.load(pose_file)["trajectory_t"]

In [33]:
global_pcd = []

for i in tqdm.trange(len(trajectory_t)):
    temp_pcd = copy.deepcopy(local_pcds[i])
    temp_pcd.transform(trajectory_t[i])
    global_pcd.append(temp_pcd)

100%|██████████| 490/490 [00:00<00:00, 2956.71it/s]


In [34]:
open3d.visualization.draw_geometries(global_pcd)

In [8]:
accel_df = pd.read_csv(os.path.join(motion_dir, "accel.csv"))
gyro_df = pd.read_csv(os.path.join(motion_dir, "gyro.csv"))

accel_df.drop_duplicates("timestamp", inplace=True)
gyro_df.drop_duplicates("timestamp", inplace=True)

gravity_df = pd.DataFrame(columns=["timestamp", "x", "y", "z"])

imu_df = pd.merge(accel_df, gyro_df, on="timestamp", suffixes=("a", "g"))

raw_accel_df = imu_df[["timestamp", "xa", "ya", "za"]].copy()

frame_rate = accel_df.shape[0] / (accel_df.timestamp.values[-1] - accel_df.timestamp.values[0]) * 1000
print(f"Frame rate: {frame_rate:.2f} Hz")

Frame rate: 400.91 Hz


In [11]:
elapsed_time = (sequence_ts - sequence_ts[0]) // 1000

In [43]:
start_t = np.argwhere(elapsed_time == 6)[0][0]
end_t = np.argwhere(elapsed_time == 8)[0][0]

In [44]:
source = local_pcds[end_t]
target = local_pcds[start_t]

trans_t = helpers.merge_transformations(start_t, end_t, local_t)

In [45]:
registration.view(source, target, trans_t)